In [ ]:
# Install TSAI for the MINIROCKET
# https://github.com/timeseriesAI/tsai
%pip install tsai --quiet

In [5]:
import polars as pl

from sklearn.model_selection import train_test_split

In [2]:
SEED = 42

In [3]:
balanced_df = pl.read_parquet("./data/balanced.pqt.zst")
display(balanced_df.sample(10, seed=SEED))

record,fs,signal,label
i64,i64,"array[f64, 640]",bool
28,128,"[-0.0204, -0.1071, … -0.204]",false
16,128,"[-0.08398, -0.0988, … 0.34086]",true
23,128,"[-0.00988, 0.01482, … 0.10374]",true
121,128,"[0.60268, 0.26676, … -0.00988]",false
23,128,"[0.40014, 0.32604, … -0.09386]",false
122,128,"[0.20748, 0.1976, … -0.5434]",true
103,128,"[-0.03952, -0.04446, … -0.06916]",true
23,128,"[-0.869441, -1.086801, … -0.17784]",false
103,128,"[0.05928, 0.10374, … -0.01482]",true


In [9]:
train_df, test_df = train_test_split(balanced_df, test_size=0.2)
val_df, test_df = train_test_split(test_df, test_size=0.5)

print(f"Train count: {train_df.shape[0]}")
print(f"Val count: {val_df.shape[0]}")
print(f"Test count: {test_df.shape[0]}")

Train count: 99920
Val count: 12490
Test count: 12490
